In [1]:
import RST_cart.generate_plane_points as points
import RST_cart.DDA as DDA
import RST_cart.physical_module as physical_module
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def cal_image(rho,pres,Te,R,wavelength,left_edge,right_edge,center,normal,width,resolution,absorption=True):
    if wave_length == 'radio':
        kappa_data, j_data = physical_module.cal_radio(rho,pres,Te)
    if wave_length == 'white_light':
        kappa_data, j_data = physical_module.cal_white_light(rho,R)
    else:
        kappa_data, j_data = physical_module.cal_euv(rho,pres,Te,wavelength,absorption)

    vc = DDA.VolumeContainer(kappa_data, j_data, left_edge, right_edge)
    
    plane_points = points.generate_plane_points(center, normal, width, resolution)
    
    image = DDA.integrate_plane_points(plane_points, normal, vc, max_t=width)

    return image

In [ ]:
import numpy as np
import time

#cgs,the sample codes from sphere to cartesian is mentioned in 

folder_path='your_path'
data=np.load(folder_path+'your_data')

rho=number_density_from_your_data
Te=temperature_from_your_data
pres=pressure_from_your_data

rho = np.nan_to_num(rho, nan=0)
Te = np.nan_to_num(Te, nan=0)
pres = np.nan_to_num(pres, nan=0)

#给定grid的边界值和选定波段
left_edge  = np.array([0.49691691,-1.21112115,-1.56216946])*7e10
right_edge = np.array([1.99999897,1.04472309,0.72845339])*7e10

#如果为EUV，填写波段的数字，如果为radio或者tomson scatter,填写字符串'radio'或者'white_light'
wavelength=304 

#EUV和radio默认光厚,white_light默认光薄,想得到光薄的EUV辐射则将absortion修改为False
absorption=True

#如果选择white_light，则需要修改下面的代码给出每个格点的半径数组,不使用white_light则R==np.zeros(rho.shape)
R==np.zeros(rho.shape)

#选定辐射合成视角
center = np.array([1.25, 0.1, -0.5])*7e10     # 平面中心
#normal = np.array([-1.50955, 1.7246, 0.3895])    # 平面法向 (y轴方向)
normal = np.array([-1, -1, 1])

#选定image射线平面足点,width表示image视线平面的宽度,resolution表示图片的分辨率（resolution*resolutino数量的射线）
width = 3*7e10
resolution = 400


image = cal_image(rho,pres,Te,R,wavelength,left_edge,right_edge,center,normal,width,resolution,absorption)


In [ ]:
import sunpy.map

plt.imshow(np.log10(image.reshape(resolution,resolution)),cmap='sdoaia304',norm=mcolors.SymLogNorm(linthresh=0.03, linscale=0.03, vmin=0, vmax=5))